# Processamento Digital de Imagens - Trabalho Prático 2

## 1. Introdução e Preparação

<p>O principal objetivo deste trabalho é <b>aplicação</b> de técnicas, embora envolva implementação também. Porém, boa parte das técnicas que necessitarão encontra-se exemplificada em algoritmos nos slides, com implementação parecida nos Notebooks disponíveis ou, às vezes, até mesmo já implementada nestes mesmos Notebooks. Portanto, o foco é no uso das técnicas, enquanto o primeiro trabalho tinha seu foco mais em implementação. Embora, mais uma vez, em ambos trabalhos estão envolvidos ambos aspectos: implementação e aplicação.</p>

### a) Edite a célula abaixo, preencha com seus dados pessoais e a execute novamente

- Aluno(a)

    - Nome: gabriel macedo nunes pontes
    - Matrícula: 98877

### b) Observações e instruções gerais (não são exatamente as mesmas do trabalho anterior, releia):

- Entregue <b>apenas</b> o notebook resultante do seu trabalho, <b>mesmo que tenha sido feito usando o Google Colab</b>. Não há necessidade de se entregar qualquer outro arquivo.
- O trabalho deve ser desenvolvido <b>inidivualmente</b> pelos alunos da UFV (Viçosa e Rio Paranaíba). Os alunos de <b>outras instituições e áreas</b> (e apenas estes), podem desenvolver em duplas.
- O trabalho é composto por várias tarefas individuais, que não são necessariamente conectadas umas às outras. Cada tarefa tem seu próprio enunciado.
- <u>A menos que seja informado o contrário no exercício</u>, está <b>vetado</b> o uso de recursos e implementações prontas dos recursos pedidos nas tarefas deste trabalho. Você deve implementar o recurso.
    - Exemplo 1: se pedido, como alvo da tarefa, que você implemente/faça a conversão de cinza de uma imagem, você não pode usar a conversão que vem pronta na biblioteca PIl.
    - Exemplo 2: se um passo secundário de um algoritmo maior demanda a conversão em cinza de uma imagem, mas não é o foco do exercício, você pode usar a conversão pronta da biblioteca PIL.
    - <font color="red">Exceção: </font> para operações morfológicas sobre imagens binárias, <u>caso você opte por usá-las</u> (não é nada obrigatório), está liberado o uso de funções prontas da biblioteca <i>OpenCV</i>
- Está <b>liberado</b> o uso de outras bibliotecas que desejar incluir, em especial, as não diretamente relacionadas ao conteúdo de PDI.
- Os exemplos mostrado ao longo deste Notebook estão na subpasta <code>imagens/exemplos/</code>. Você pode abri-las separadamente, para analisar em mais detalhes, se desejar.

### c) Execute a célula abaixo, que conterá as importações de módulos Python.

#### Você é livre para (<font color="red">e provavelmente precisará</font>) editar e adicionar outras importações a esta célula.

In [ ]:
# Plotagem de gráficos e imagens no notebook
%matplotlib inline
import matplotlib.pyplot as plt

# Manipulação de imagens
from PIL import Image
import cv2
from scipy.interpolate import interp2d
import numpy as np # Manipulação de arrays
import os # Acesso ao sistema de arquivos do SO


### d) Funções auxiliares prontas. Execute as células a seguir.

<p>Função <code>exibe_bloco</code>:</p>

- Exibe, em tela, diversas imagens, em um grid
- <u>Parâmetros de entrada:</u>
    - <code>imgs</code>: lista <i>Python</i> contendo $n > 1$ objetos <code>PIL Image</code>, a serem exibidos em tela
    - <code>titulos</code>: lista <i>Python</i> contendo $n$ strings, com o título de cada imagem. Opcional. Se não fornecida, gera títulos automaticamente, com 'Imagem 1', 'Imagem 2' etc.
    - <code>tam_linha</code>: valor inteiro, informando quantas imagens por linha serão exibidas. Opcional. Valor <i>default</i>: 4
    - <code>arquivo</code>: string que, se fornecida, habilita o salvamento da visualização das imagens em um arquivo em memória secundária, usando o nome de arquivo fornecido no parâmetro. Opcional. <i>Default:</i> <code>None</code>
    - <code>dpi</code>: <i>float</i> que, se fornecido, determina a resolução da imagem gerada. Opcional. <i>Default</i>: $100$
    - Gera erro se os comprimentos de <code>imgs</code> e <code>titulos</code> não forem os mesmos ou se <code>len(imgs)</code> $< 2$.
- <u>Retorno:</u> nenhum

In [ ]:
def exibe_bloco(imgs, titulos=None, tam_linha=4, arquivo=None, dpi=100):
    assert len(imgs) > 1
    
    # Espaçamento entre imagens (em pixels)
    espacamento_x = 15
    espacamento_y = 20
    
    # Calcula o número de linhas
    if len(imgs) < tam_linha:
        tam_linha = len(imgs)
    nlinhas = len(imgs) // tam_linha + int(len(imgs) % tam_linha > 0)
    
    # Cria lista de arrays e calcula tamanho da imagem de visualização a ser exibida/salva
    largura_px = 0
    altura_px = 0
    i = 0
    imagens = []
    for img in imgs:
        if img.mode == 'L':
            img = img.convert('RGB')
        imagens.append(np.asarray(img))
        if i % tam_linha == 0:
            max_altura_linha = 0
            largura_linha = 0
        largura_linha += img.width
        if img.height > max_altura_linha:
            max_altura_linha = img.height
        if i % tam_linha == tam_linha-1 or i == len(imgs)-1:
            altura_px += max_altura_linha
            if largura_linha > largura_px:
                largura_px = largura_linha
        i += 1        
    largura_px += (tam_linha-1)*espacamento_x
    altura_px += (nlinhas-1)*espacamento_y
    largura_inch = largura_px / float(dpi)
    altura_inch = altura_px / float(dpi)
    
    # Cria lista de títulos
    if titulos is None:
        titulos = []
        for i in range(len(imagens)):
            titulos.append('Imagem %d' % (i+1))
    else:
        assert len(imgs) == len(titulos)
    
    # Plota imagens em grid
    plt.rcParams['figure.figsize'] = [largura_inch,altura_inch]
    fig, axes = plt.subplots(nrows=nlinhas, ncols=tam_linha)
    for i,ax in enumerate(axes.ravel()):
        ax.axis('off')
        if i < len(imgs):
            ax.imshow(imagens[i])
            ax.set_title(titulos[i])
        else:
            fig.delaxes(ax)
        
    fig.tight_layout()
    
    # Salva, se desejado
    if arquivo is not None:
        plt.savefig(arquivo, dpi=dpi)
        print('Visualização salva em %s.' % arquivo)
        
    # Exibe
    plt.show()

<p>Função <code>getArrayCinza</code>:</p>

- Retorna um array <code>numpy</code> contendo as informações da versão em tons de cinza de uma imagem
- Se a entrada for uma imagem colorida, faz a conversão para tons de cinza antes de extrair o array, ou, se a entrada já for monocromática, apenas extrai e retorna o array
- <u>Parâmetro de entrada:</u> <code>img</code>: objeto <code>PIL Image</code>
- <u>Retorno:</u> array <code>numpy</code>

In [ ]:
def getArrayCinza(img):
    if img.mode == 'L':
        arr = np.asarray(img).copy()
    else:
        arr = np.asarray(img.convert('L')).copy()
    return arr

## 2. Recuperação de informações em imagens de baixa qualidade

<p>Para todos exercícios desta parte, você é livre para usar <b>quaisquer</b> conhecimentos vistos na disciplina e pode/deve combinar técnicas à vontade.</p>

<p><b>Obs.:</b> diferentemente do primeiro trabalho, desta vez as imagens são apenas um guia ilustrativo, uma referência à qual você deve se aproximar o máximo que conseguir. No entanto, poderão ser consideradas como "corretas", respostas que se difiram dos exemplos ilustrados em cada item.</p>

### 2.1 Tornando textos mais legíveis

<p>Na subpasta <code>imagens</code> encontram-se três fotos de textos impressos em papel, obtidas sob más condições de iluminação ou de sombra e uma imagem "sintética", de um texto em formato digital sobre o qual foi inserida uma sombra artificialmente. São os arquivos:</p>

- <code>texto_1.jpg</code>
- <code>texto_2.jpg</code>
- <code>texto_sombra.jpg</code>
- <code>texto_sombra_sintetico.jpg</code>

<p>Copie e cole a implementação do algoritmo de Otsu dado em aula e binarize as imagens com a função, para ver os resultados. Responda na célula abaixo: o que aconteceu com o resultado? Ele é útil, na prática? Por que você acha que aconteceu este fenômeno?</p>

In [ ]:
def histograma(f, L):
    h = np.zeros(L)
    
    for k in range(L):
        h[k] = np.sum(f == k)
    
    # O laço acima equivale a:
    #for y in range(f.shape[0]):
    #    for x in range(f.shape[1]):
    #        k = int(f[y,x])
    #        h[k] += 1
        
    return h

def calcula_cdf(p):
    cdf = np.zeros(p.shape)
    
    for k in range(len(p)):
        cdf[k] = np.sum(p[:k+1])
        
    return cdf

def gera_histogramas(f, L):
    # Contagem de pixels da imagem
    n = f.shape[0] * f.shape[1]
    
    h = histograma(f, L)
    p = h / n
    cdf = calcula_cdf(p)

    return h,p,cdf
    
def histogramas(f, Lmax=255):
    # Contagem do número de tons:
    L = Lmax + 1
    
    # identifica o tipo de imagem
    if len(f.shape) < 3:
        # Imagem cinza
        return gera_histogramas(f, L)
    else:
        # Imagem RGB
        hr,pr,cdfr = gera_histogramas(f[:,:,0], L)
        hg,pg,cdfg = gera_histogramas(f[:,:,1], L)
        hb,pb,cdfb = gera_histogramas(f[:,:,2], L)
        
        return (hr,hg,hb), (pr,pg,pb), (cdfr, cdfg, cdfb)

In [ ]:
def otsu(img, verboso=False):
    # Converte para tons de cinza, se necessário, e extrai array
    if img.mode != 'L':
        arr = np.asarray(img.convert('L')).copy()
    else:
        arr = np.asarray(img).copy()
        
    # Calcula histogramas
    p = histograma(arr, 256) / (img.width * img.height)
    cdf = calcula_cdf(p)
    
    # Calcula média geral
    i = np.arange(0, 256) # Array com os valores dos tons de cinza
    mu = np.sum(p * i)
    
    # Inicializa variáveis de controle
    th = 0
    maxVar = 0
    
    # Encontra o tom k que leva à máxima variância intergrupos
    for k in range(1,255):
        # Calcula as médias dos valores de cada grupo gerado com k como limiar
        if cdf[k] != 0:
            mu1 = np.sum(p[:k+1] * i[:k+1]) / cdf[k]
        else:
            mu1 = 0.0
        
        if cdf[k] != 1:
            mu2 = np.sum(p[k+1:] * i[k+1:]) / (1 - cdf[k])
        else:
            mu2 = 0.0
        
        # Calcula a variância intergrupos
        sigma2 = cdf[k] * (mu1 - mu)** 2 + (1 - cdf[k]) * (mu2 - mu)**2
        
        # Se a variância integrupos da iteração é o novo valor máximo, atualiza
        if sigma2 > maxVar:
            maxVar = sigma2
            th = k
            
    # Aplica a binarização com o valor de tom que leva ao máximo de variância encontrado (salvo em th)
    arr[arr <= th] = 0
    arr[arr > th] = 255
    
    if verboso:
        print('Limiar ótimo encontrado: %.4f' % th)
    
    return Image.fromarray(np.uint8(arr))

In [ ]:
texto1 = Image.open('imagens/texto_1.jpg')
texto1=otsu(texto1)
texto2 = Image.open('imagens/texto_2.jpg')
texto2=otsu(texto2)
textos = Image.open('imagens/texto_sombra.jpg')
textos=otsu(textos)
textoss = Image.open('imagens/texto_sombra_sintetico.jpg')
textoss=otsu(textoss)
img=[texto1,texto2,textos,textoss]
exibe_bloco(img)

<p><font color="red">Adicione, a seguir, quantas células desejar e implemente</font> códigos que melhorem a legibilidade dos textos contidos nos arquivos. Leve em conta:</p>

- Pode-se aproveitar soluções já feitas por você ou que tenha usado no trabalho anterior, se desejar
- O resultado final <b>deve</b> estar em formato "pseudo-binário", isto é, imagens em tons de cinza, mas contendo somente pixels com valores $0$ e $255$
- Existem diversas maneiras de se obter resultados como os mostrados nos exemplos, portanto, conforme já dito, os exemplos são apenas para se mostrar que tipo de resultado obter
- As soluções dadas por você podem ser específicas para cada caso, não é necessário se criar uma função que funcione para os quatro casos dados. Por exemplo:
    - A escolha de parâmetros de algoritmos para um caso pode ser diferente da de outro
    - A escolha do encademento/sequência de técnicas usadas para um caso pode ser diferente da de outro
- Gere pelo menos uma solução para cada caso. Se conseguir gerar mais, melhor :-)

<p>Exemplos:</p>

<img src="imagens/exemplos/texto_1.jpg?2" width="100%">
<img src="imagens/exemplos/texto_2.jpg?2" width="100%">
<img src="imagens/exemplos/texto_sombra.jpg?2" width="100%">
<img src="imagens/exemplos/texto_sombra_sintetico.jpg?2" width="100%">

- Para produzir cada item de exemplo acima, foram utilizadas duas sequências de diferentes passos, como filtro de suavização, melhorias de contraste, limiarizações etc
- As imagens resultados encontram-se na subpastas <code>imagens/exemplos/</code>, as quais podem ser visualizadas separadamente, com a possibilidade de zoom e outras operações que desejar, para analisar os resultados produzidos

In [ ]:
def equaliza(img):
    p = histograma(img, 256) / (img.shape[0] * img.shape[1])
    cdf = calcula_cdf(p)*255
    for X in range(img.shape[0]):
        for Y in range(img.shape[1]):
            pixel=img[X,Y]
            pixel=cdf[pixel]
            img[X,Y]=pixel
    return img
                
def limiarVariavel(arr, t, a=0.5, b=0.5):
    # A fazer: vetar t de valor par
    raio = t//2
    
    # Array de retorno
    arrBin = arr.copy()
    
    # Percorre a imagem pixel a pixel calculando os limiares locais
    for y in range(arr.shape[0]):
        iniVizy = max(0, y-raio)
        fimVizy = min(y+raio, arr.shape[0]-1)
        
        for x in range(arr.shape[1]):
            iniVizx = max(0, x-raio)
            fimVizx = min(x+raio, arr.shape[1]-1)
            
            mu = np.mean(arr[iniVizy:fimVizy+1, iniVizx:fimVizx+1])
            sigma = np.std(arr[iniVizy:fimVizy+1, iniVizx:fimVizx+1])
            
            th = a * sigma + b * mu
            
            if arr[y,x] <= th:
                arrBin[y,x] = 0
            else:
                arrBin[y,x] = 255
                
    return Image.fromarray(np.uint8(arrBin))

In [ ]:
texto1 = Image.open('imagens/texto_1.jpg')
texto2 = Image.open('imagens/texto_2.jpg')
textos = Image.open('imagens/texto_sombra.jpg')
textoss = Image.open('imagens/texto_sombra_sintetico.jpg')
texto1=getArrayCinza(texto1)
texto2=getArrayCinza(texto2)
textos=getArrayCinza(textos)
textoss=getArrayCinza(textoss)
texto1=limiarVariavel(texto1,14)
texto1=Image.fromarray(np.uint8(texto1))
texto2=limiarVariavel(texto2,14)
texto2=Image.fromarray(np.uint8(texto2))
textos=equaliza(textos)
textos=limiarVariavel(textos,14)
textos=Image.fromarray(np.uint8(textos))
textoss=limiarVariavel(textoss,14,a=0.5,b=0.9)
textoss=Image.fromarray(np.uint8(textoss))
exibe_bloco([texto1,texto2,textos,textoss])

### 2.2 Recuperando informação de foto com problemas de exposição

<p>Na tentativa de se obter uma foto como a da esquerda, no exemplo abaixo, usando uma câmera semiprofissional com um filtro de lente especial, cometeu-se um erro nas configurações da câmera e a imagem danificada, na direita, foi gerada. Não se trata de um exemplo simulado, mas um exemplo real de uma foto que foi adquirida com defeitos.</p>

<img src="imagens/exemplos/copos.png" width="80%">

<p>O arquivo da foto defeituosa encontra-se em <code>imagens/vermelho.png</code>.</p>

<p>Mesmo com os problemas da foto defeituosa, ainda é possível se destacar, mesmo que de forma imprecisa, algumas informações de forma dos objetos principais presentes na mesma.</p>

<p><font color="red">Adicione, a seguir, quantas células desejar e implemente</font> códigos que visam destacar, de alguma forma os objetos principais da imagem danificada. Leve em conta:</p>

- Pode-se aproveitar soluções já feitas por você ou que tenha usado no trabalho anterior, se desejar
- O resultado final <b>pode</b> estar em formato de tons de cinza propriamente dito ou em formato "pseudo-binário", isto é, imagens em tons de cinza, mas contendo somente pixels com valores $0$ e $255$
- Existem diversas maneiras de se obter resultados como os mostrados nos exemplos, portanto, conforme já dito, os exemplos são apenas para se mostrar que tipo de resultado obter
- As soluções dadas por você podem ser específicas para cada caso, não é necessário se criar uma função que funcione para os quatro casos dados. Por exemplo:
    - A escolha de parâmetros de algoritmos para um caso pode ser diferente da de outro
    - A escolha do encademento/sequência de técnicas usadas para um caso pode ser diferente da de outro
- Gere pelo menos uma solução para cada caso. Se conseguir gerar mais, melhor :-)

<p>Exemplos de resultados:</p>

<img src="imagens/exemplos/copos_destaque.png" width="80%">

In [ ]:
def convolucao_linear_impar(img, k, canais, clip=True, ret_img=True):
    # Centro do kernel e deslocamento
    desloc = i = j = k.shape[0] // 2
    
    # Cria array com padding
    if canais > 1:
        arr = np.zeros((img.height+2*desloc, img.width+2*desloc, canais))
    else:
        arr = np.zeros((img.height+2*desloc, img.width+2*desloc))
    # Copiar dados da imagem para o "centro" do array com padding
    if canais > 1:
        arr[desloc:arr.shape[0]-desloc,desloc:arr.shape[1]-desloc,:] = np.asarray(img)
    else:
        arr[desloc:arr.shape[0]-desloc,desloc:arr.shape[1]-desloc] = np.asarray(img)
    arr_orig = arr.copy()
    
    # Efetua convolução (tamanho ímpar, com referência no centro do kernel)
    # Percorre apenas posições "internas"
    for y in range(desloc,arr.shape[0]-desloc):
        for x in range(desloc,arr.shape[1]-desloc):
            # Sobreposição do kernel sobre a imagem
            if canais == 1:
                prod = arr_orig[y-desloc:y+desloc+1,x-desloc:x+desloc+1]*k[i-desloc:i+desloc+1,j-desloc:j+desloc+1]
                arr[y,x] = np.sum(prod)
            else:
                for c in range(canais):
                    prod = arr_orig[y-desloc:y+desloc+1,x-desloc:x+desloc+1,c]*k[i-desloc:i+desloc+1,j-desloc:j+desloc+1]
                    arr[y,x,c] = np.sum(prod)
            
    # Criação do array resultante
    arr = np.round(arr)
    if clip:
        arr = np.clip(arr, 0, 255)
    
    # Extrai somente a parte "central" da imagem
    arr = arr[desloc:arr.shape[0]-desloc, desloc:arr.shape[1]-desloc]
    
    if ret_img:
        arr = np.uint8(arr)
        return Image.fromarray(arr)
    else:
        return arr

def convolucao_linear_par(img, k, canais, clip=True, ret_img=True):
    tam_k = k.shape[0]
    
    # Cria array com padding
    if canais > 1:
        arr = np.zeros((img.height+tam_k, img.width+tam_k, canais))
    else:
        arr = np.zeros((img.height+tam_k, img.width+tam_k))
    # Copia dados da imagem para a "parte inicial" do array com padding
    if canais > 1:
        arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k, :] = np.asarray(img)
    else:
        arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k] = np.asarray(img)
    arr_orig = arr.copy()
    
    # Efetua convolução (tamanho par, com referência no pixel superior esquerdo do kernel)
    # Percorre apenas posições "iniciais"
    for y in range(0,arr.shape[0]-tam_k):
        for x in range(0,arr.shape[1]-tam_k):
            # Sobreposição do kernel sobre a imagem
            if canais == 1:
                prod = arr_orig[y:y+tam_k,x:x+tam_k]*k[:tam_k,:tam_k]
                arr[y,x] = np.sum(prod)
            else:
                for c in range(canais):
                    prod = arr_orig[y:y+tam_k,x:x+tam_k,c]*k[:tam_k,:tam_k]
                    arr[y,x,c] = np.sum(prod)
    
    # Criação do array resultante
    arr = np.round(arr)
    if clip:
        arr = np.clip(arr, 0, 255)
    
    # Extrai somente a parte "central" da imagem
    arr = arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k]
    
    if ret_img:
        arr = np.uint8(arr)
        return Image.fromarray(arr)
    else:
        return arr

def convolucao_linear(img, k, clip=True, ret_img=True):
    # Determina o número de canais
    if img.mode == 'RGB' or img.mode == 'RGBA':
        canais = 3
    else:
        canais = 1
        
    if k.shape[0] % 2 == 1:
        return convolucao_linear_impar(img,k,canais,clip,ret_img)
    else:
        return convolucao_linear_par(img,k,canais,clip,ret_img)
    
def convolucao_impar(img, operacao, tam_k, canais, clip=True, ret_img=True):
    # Centro do kernel e deslocamento
    desloc = i = j = tam_k // 2
    
    # Cria array com padding
    if canais > 1:
        arr = np.zeros((img.height+2*desloc, img.width+2*desloc, canais))
    else:
        arr = np.zeros((img.height+2*desloc, img.width+2*desloc))
    # Copiar dados da imagem para o "centro" do array com padding
    if canais > 1:
        arr[desloc:arr.shape[0]-desloc,desloc:arr.shape[1]-desloc,:] = np.asarray(img)
    else:
        arr[desloc:arr.shape[0]-desloc,desloc:arr.shape[1]-desloc] = np.asarray(img)
    arr_orig = arr.copy()
    
    # Efetua convolução (tamanho ímpar, com referência no centro do kernel)
    # Percorre apenas posições "internas"
    for y in range(desloc,arr.shape[0]-desloc):
        for x in range(desloc,arr.shape[1]-desloc):
            # Sobreposição do kernel sobre a imagem
            if canais > 1:
                arr[y,x,:] = operacao(arr_orig[y-desloc:y+desloc+1,x-desloc:x+desloc+1,:])
            else:
                arr[y,x] = operacao(arr_orig[y-desloc:y+desloc+1,x-desloc:x+desloc+1])
    
    # Criação do array resultante
    arr = np.round(arr)
    if clip:
        arr = np.clip(arr, 0, 255)
    
    # Extrai somente a parte "central" da imagem
    arr = arr[desloc:arr.shape[0]-desloc, desloc:arr.shape[1]-desloc]
    
    if ret_img:
        arr = np.uint8(arr)
        return Image.fromarray(arr)
    else:
        return arr

def convolucao_par(img, operacao, tam_k, canais, clip=True, ret_img=True):
    # Cria array com padding
    if canais > 1:
        arr = np.zeros((img.height+tam_k, img.width+tam_k, canais))
    else:
        arr = np.zeros((img.height+tam_k, img.width+tam_k))
    # Copia dados da imagem para a "parte inicial" do array com padding
    if canais > 1:
        arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k, :] = np.asarray(img)
    else:
        arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k] = np.asarray(img)
    arr_orig = arr.copy()
    
    # Efetua convolução (tamanho par, com referência no pixel superior esquerdo do kernel)
    # Percorre apenas posições "iniciais"
    for y in range(0,arr.shape[0]-tam_k):
        for x in range(0,arr.shape[1]-tam_k):
            # Sobreposição do kernel sobre a imagem
            if canais > 1:
                arr[y,x,:] = operacao(arr_orig[y:y+tam_k,x:x+tam_k,:])
            else:
                arr[y,x] = operacao(arr_orig[y:y+tam_k,x:x+tam_k])
    
    # Criação do array resultante
    arr = np.round(arr)
    if clip:
        arr = np.clip(arr, 0, 255)
    
    # Extrai somente a parte "central" da imagem
    arr = arr[:arr.shape[0]-tam_k, :arr.shape[1]-tam_k]
    
    if ret_img:
        arr = np.uint8(arr)
        return Image.fromarray(arr)
    else:
        return arr

# Convolução "genérica": recebe função como parâmetro
def convolucao(img, operacao, tam_k, clip=True, ret_img=True):
    # Determina o número de canais
    if img.mode == 'RGB' or img.mode == 'RGBA':
        canais = 3
    else:
        canais = 1
        
    if tam_k % 2 == 1:
        return convolucao_impar(img, operacao, tam_k, canais, clip, ret_img)
    else:
        return convolucao_par(img, operacao, tam_k, canais, clip, ret_img)

In [ ]:
img = Image.open('imagens/vermelho.png')
img=getArrayCinza(img)
img=Image.fromarray(equaliza(img))
img.show()

### 2.3 Realce em imagem embassada

<p>A imagem abaixo foi adquirida, intencionalmente, para estar desfocada, a fim de emular um defeito de aquisição.</p>

<img src="imagens/ufv_desfocado.jpg" width="40%">

<p>O arquivo da foto defeituosa encontra-se em <code>imagens/ufv_desfocado.jpg</code>.</p>

<p>Deseja-se amenizar o problema da identificação de forma, usando algum tipo de processamento sobre a mesma.</p>

<p><font color="red">Adicione, a seguir, quantas células desejar e implemente</font> código(s) que visa(m) destacar as formas dos objetos borrados por meio do filtro generalizado de nitidez, visto em aula. Leve em conta:</p>

- Pode-se aproveitar soluções já feitas por você ou que tenha usado no trabalho anterior, se desejar
- O resultado final <b>deve</b> estar em formato colorido RGB
- Teste diferentes valores para o parâmetro $k$, do filtro
- A imagem original já se encontra desfocada, então certos filtros de suavização (usados como uma etapa do highboost) podem não produzir um bom resultado. Teste diferentes filtros, incluindo os não lineares.
- Produza um resultado para <i>Highboost</i> e um para realce atenuado, conforme descrito em aula e no material

<p><b>Exemplo de resultado:</b></p>

<p>A primeira versão é um caso de <i>Highboost</i>. A segunda versão é uma implementação usando um filtro de realce atenuado e um método de suavização diferente do primeiro. Note que a mudança do filtro de suavização também influencia no resultado</p>

<img src="imagens/exemplos/highboost.png?2" width="100%">

In [ ]:
def _mediana(M):
    if len(M.shape) == 3:
        ret = []
        for c in range(M.shape[2]):
            ret.append(np.median(M[:,:,c]))
        return np.array(ret)
    else:
        return np.median(M)
    
def _maximo(M):
    if len(M.shape) == 3:
        ret = []
        for c in range(M.shape[2]):
            ret.append(np.max(M[:,:,c]))
        return np.array(ret)
    else:
        return np.max(M)
def suavizacao_mediana(img, tam_k):
    return convolucao(img, _mediana, tam_k)

def suavizacao_maximo(img, tam_k):
    return convolucao(img, _maximo, tam_k)
def suavizacao_gauss(img, tam_k, sigma=1):
    # Lança exceção, se tamanho do filtro for par
    if tam_k % 2 == 0:
        raise Exception('\n\n\nErro!! O tamanho do kernel deve ser ímpar!!\n\n\n')
    
    # Cria kernel
    x = np.arange(-(tam_k-1)/2, (tam_k-1)/2+1)
    y = np.arange(-(tam_k-1)/2, (tam_k-1)/2+1)

    h1,h2 = np.meshgrid(x,y)

    hg = np.exp(-(h1**2 + h2**2) / (2*sigma**2))
    h = hg / np.sum(hg)

    # Retorna convolução
    return convolucao_linear(img,h)

In [ ]:
imgOrig=Image.open('imagens/ufv_desfocado.jpg')
arrOrig = np.asarray(imgOrig, dtype=float)
suavehigh=suavizacao_mediana(imgOrig,3)
arrSuavehigh = np.asarray(suavehigh, dtype=float)
high = np.uint8(np.clip(arrOrig + 4*(arrOrig - arrSuavehigh), 0, 255))
imghigh = Image.fromarray(high)
suaveatenuado=suavizacao_maximo(imgOrig,9)
arrSuaveatenuado = np.asarray(suaveatenuado, dtype=float)
atenuado = np.uint8(np.clip(arrOrig + 0.5*(arrOrig - arrSuaveatenuado), 0, 255))
imgatenuado = Image.fromarray(atenuado)
exibe_bloco([imgOrig,imghigh,imgatenuado], ['Original','imghigh','atenuado'])

## 3. Filtro de Gradiente

<p><font color="red">Adicione, a seguir, quantas células desejar de desenvolva</font> código(s) que implemente(m) uma versão do <b>filtro de Canny</b>, visto em aula. Leve em conta:</p>

- Pode-se aproveitar soluções já feitas por você ou que tenha usado no trabalho anterior, se desejar
- O resultado final <b>deve</b> estar em formato colorido "pseudo-binário", conforme já descrito
- As imagens de entrada nos exemplos estão na pasta <code>imagens</code> e são: <code>ipe.jpg</code>, <code>mtstmichel.jpg</code> e <code>ufv.png</code>.

<p><b>Exemplos de resultado:</b></p>

<img src="imagens/exemplos/ipe.jpg" width="80%">
<img src="imagens/exemplos/mtstmichel.jpg" width="80%">
<img src="imagens/exemplos/ufv.png" width="80%">

In [ ]:
def filtro_combinado(img, kernels):
    k1 = np.array(kernels[0])
    k2 = np.array(kernels[1])
    primeiro=convolucao_linear(img, k1, clip=False, ret_img=False)
    segundo=convolucao_linear(img, k2, clip=False, ret_img=False)
    return primeiro,segundo
def sobel(img):
    k1 = [[-1, -2, -1],
          [0, 0, 0],
          [1, 2, 1]]
    k2 = [[-1, 0, 1],
          [-2, 0, 2],
          [-1, 0, 1]]
    
    return filtro_combinado(img, [k1,k2])
def supressao(G,angulo):
    angulo=angulo*(180/3.14)
    angulo[angulo<0]+=180
    z=np.zeros_like(G,dtype=float)
    for i in range(1,G.shape[0]-1):
        for j in range(1,G.shape[1]-1):
            q=255
            r=255
            if 0<=angulo[i][j]<22.5 or 157.5<=angulo[i][j]<=180:
                q=G[i,j+1]
                r=G[i,j-1]
            elif 22.5<=angulo[i][j]<67.5:
                q=G[i+1,j-1]
                r=G[i-1,j+1]
            elif 67.5<=angulo[i][j]<112.5:
                q=G[i+1,j]
                r=G[i-1,j]
            elif 112.5<=angulo[i][j]<157.5:
                q=G[i-1,j-1]
                r=G[i+1,j+1]
            if G[i,j]>=q and G[i,j]>=r:
                z[i,j]=G[i,j]
    return z
def duplo(z,al,be):
    r=np.zeros_like(z,dtype=float)
    b=be*np.amax(z)
    a=al*b
    f=np.round(b)+1
    F=255
    r[z>=b]=F
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            if a<=z[i,j]<b:
                r[i,j]=f
    return r,f,F
def final(r,f,F):
    for i in range(1,r.shape[0]-1):
        for j in range(1,r.shape[1]-1):
            if r[i,j]==f:
                for k in range(-1,2):
                    for c in range(-1,2):
                        if r[k,c]==F:
                            r[i,j]=F
                            break;
                if r[i,j]==f:
                    r[i,j]=0
    return r

In [ ]:
img=Image.open('imagens/ipe.jpg')
img=getArrayCinza(img)
img=Image.fromarray(img)
retorno1,retorno2=sobel(img)
G = np.clip(np.sqrt((retorno1**2)+(retorno2**2)), 0, 255)
angulo=np.zeros_like(retorno1,dtype=float)
np.arctan2(retorno1,retorno2,angulo)
z=supressao(G,angulo)
r,f,F=duplo(z,0.1,0.6)
ipe=final(r,f,F)
ipe=Image.fromarray(ipe)
img=Image.open('imagens/mtstmichel.jpg')
img=getArrayCinza(img)
img=Image.fromarray(img)
retorno1,retorno2=sobel(img)
G = np.clip(np.sqrt((retorno1**2)+(retorno2**2)), 0, 255)
angulo=np.zeros_like(retorno1,dtype=float)
np.arctan2(retorno1,retorno2,angulo)
z=supressao(G,angulo)
r,f,F=duplo(z,0.1,0.6)
mt=final(r,f,F)
mt=Image.fromarray(mt)
img=Image.open('imagens/ufv.png')
img=getArrayCinza(img)
img=Image.fromarray(img)
retorno1,retorno2=sobel(img)
G = np.clip(np.sqrt((retorno1**2)+(retorno2**2)), 0, 255)
angulo=np.zeros_like(retorno1,dtype=float)
np.arctan2(retorno1,retorno2,angulo)
z=supressao(G,angulo)
r,f,F=duplo(z,0.1,0.6)
ufv=final(r,f,F)
ufv=Image.fromarray(ufv)
ipe.show()
mt.show()
ufv.show()
exibe_bloco([ipe,mt,ufv], ['ipe','mt','ufv'])

## 4. Filtro "Artístico"

<p><font color="red">Adicione, a seguir, quantas células desejar de desenvolva</font> código(s) que implemente(m) um filtro artístico com aspecto de "desenho", como os dos exemplos a seguir. Algumas soluções similares estão nos slides do material de aula.</p>

<p>Note que passos comuns envolvem detecção de bordas, saturação de cores, filtros de suavização que uniformizam cores/regiões, obtenção de negativos e filtros espaciais em geral. Esteja livre para usar quaisquer filtros ou técnicas que desejar. Pode-se aproveitar implementações de sala de aula ou do trabalho anterior, para agilizar.</p>

<p><b>Importante:</b> <u>para esta atividade apenas</u>, você pode utilizar implementações prontas de filtros de bibliotecas de imagem em Python, como <code>PIL</code>, <code>OpenCV</code>, <code>skimage</code>, <code>ndimage</code> e afins.</p>

<p>As imagens utilizadas nos exemplos encontram-se na pasta <code>imagens/</code> e são: <code>ipe.jpg</code>, <code>mtstmichel.jpg</code>, <code>ufv.png</code> e <code>praiaPessoas.png</code>.

<p><b>Exemplos de resultado:</b></p>

<img src="imagens/exemplos/ipe_desenho.jpg" width="100%">
<img src="imagens/exemplos/mtstmichel_desenho.jpg" width="100%">
<img src="imagens/exemplos/ufv_desenho.png" width="100%">
<img src="imagens/exemplos/praiaPessoas_desenho.png" width="100%">

In [ ]:
imgorig=Image.open('imagens/ufv.png')
img=getArrayCinza(imgorig)
gauss= cv2.GaussianBlur(src=img, ksize=(5,5),sigmaX=0, sigmaY=0)
edges = cv2.adaptiveThreshold(gauss, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,3,3)
chapada=np.asarray(imgorig)
chapada=cv2.medianBlur(chapada,11)
chapada=np.asarray(chapada)
saida =cv2.bitwise_and(chapada, chapada, mask=-edges)
saida=img=Image.fromarray(saida)
saida.show()

## 5. Transformação geométrica

<p><font color="red">Complete o código da função</font> <code>redim</code>, para que a célula de código de teste funcione, apresentando os resultados desejados.</p>

<p>A função <code>redim</code> deve receber uma imagem e um fator de proporção como entrada e aplicar este fator para redimensionar a imagem. Por exemplo, se o fator for $0,3$, a imagem resultante terá $30\%$ do tamanho da original, proporção esta aplicada tanto sobre a altura quanto sobre a largura da imagem original. Se o fator for igual a $1$, a imagem original não é alterada, enquanto que se o fator for superior a $1$, a imagem será ampliada. Por exemplo, para um fator de $1,4$, a imagem resultante será $40\%$ maior que a original</p>

<p>A imagem usada para a célula de teste está na pasta de imagens e é o arquivo <code>capivara.jpg</code>. No entanto, vale você testar com outras das imagens também, para testar se sua implementação funciona para casos gerais.</p>

<p><b>Importante:</b> Deste ponto em diante está novamente <b>vetado</b> o uso de funções prontas de bibliotecas Python para imagens.</p>

In [ ]:
def redim(imagem, fator):
    img=np.asarray(imagem)
    Y=np.round(fator*img.shape[0])
    X=np.round(fator*img.shape[1])
    y=np.arange(0,img.shape[0])
    x=np.arange(0,img.shape[1])
    yredu=np.linspace(0,img.shape[0]-1,int(Y))
    xredu=np.linspace(0,img.shape[1]-1,int(X))
    saida=np.empty((int(Y),int(X),3),dtype=np.uint8)
    interred=interp2d(x,y,img[:,:,0])
    intergreen=interp2d(x,y,img[:,:,1])
    interblue=interp2d(x,y,img[:,:,2])
    saida[:,:,0]=np.round(interred(xredu,yredu)).astype(np.uint8)
    saida[:,:,1]=np.round(intergreen(xredu,yredu)).astype(np.uint8)
    saida[:,:,2]=np.round(interblue(xredu,yredu)).astype(np.uint8)
    return Image.fromarray(saida)

### Célula de teste

In [ ]:
img = Image.open('imagens/capivara.jpg')
grande = redim(img, 1.5)
pequena = redim(img, 0.3)
img.show()
grande.show()
pequena.show()

## 6. Detecção de bordas em imagem escura

<p>Dada a imagem do arquivo <code>dpi_escuro.png</code>, <font color="red">adicione quantas células desejar a seguir</font> e implemente um código que, mesmo na imagem sob as más condições de luz, seja capaz de identificar potenciais regiões de borda na imagem, conforme ilustrado nas imagens da figura a seguir:</p>

<img src="imagens/exemplos/corredor.png?2" width="100%">

<p>Uma possível aplicação para este tipo de cenário seria auxiliar um robô, cuja navegação é orientada por meio de uma câmera, a identificar regiões que delimitam objetos para se desviar deles, mesmo com pouca luz.</p>

<p>O importante do exercício é identificar as tais potenciais bordas, obtidas a partir da primeira imagem, cujo arquivo é mencionado anteriormente. Ou seja, a <b>terceira</b> (inferior, à esquerda) imagem do exemplo acima é o objetivo. Esta que deverá ser sua resposta ao exercício proposto.</p>

<p>A segunda imagem é meramente ilustrativa e é uma outra foto, adquirida separadamente e no mesmo ambiente, porém sujeita a melhores condições de iluminação. Está presente na figura do exemplo apenas para que se verifique a qualidade da detecção das bordas. Ela <u>não</u> é uma versão da imagem original e nem será utilizada, em momento algum, para o desenvolvimento da solução do exercício.</p>

<p>Por fim, quarta imagem também é apenas para ilustração. Neste caso, das bordas encontradas superpostas à imagem original, escura. Não é exigida como parte da sua solução. No entanto, se você fizer esta quarta imagem, receberá pontos bônus neste exercício...</p>

In [ ]:
img = Image.open('imagens/dpi_escuro.png')
cinza=img.convert('L')
gauss= suavizacao_gauss(cinza,11)
gauss=np.asarray(gauss)
equa=equaliza(gauss)
equa=Image.fromarray(equa)
retorno1,retorno2=sobel(equa)
G = np.clip(np.sqrt((retorno1**2)+(retorno2**2)), 0, 255)
angulo=np.zeros_like(retorno1,dtype=float)
np.arctan2(retorno1,retorno2,angulo)
z=supressao(G,angulo)
r,f,F=duplo(z,0,0.7)
r=final(r,f,F)
r=Image.fromarray(np.uint8(np.round(r)))
r.show()

## Considerações finais

- Salve seu Notebook e envie, pelo Moodle, apenas ele.
- Releia as observações no início do enunciado para checar se não se esqueceu de nenhum item
- Confira seus resultados com os exemplos fornecidos, mas também faça testes criados por você

<p style="text-align: right">Bons estudos,</p>
<p style="text-align: right">Prof. Marcos</p>